# **Spike Triggered Average – STA**

We have already looked into how to handle and analyze spike trains recorded under visual stimulation. We will now turn to a very common tool in systems neuroscience, the spike triggered average. 

**(here goes the theory, if not in a separate presentation)**


**You will learn to:**
-  Match the signal from the stimulus generator to the actual presentation of the stimulus.
-  Collect the stimulus portions preceding each spike, given a certain time window.
-  Find the average stimulus that evokes spiking.
-  **(simulation, fitting, shuffle analysis?)**

First, let's import relevant packages:

In [ ]:
%matplotlib inline
%config InlineBackend.rc={'figure.figsize': (12, 6), 'font.size': 14 }
from matplotlib import pyplot as plt
import numpy as np
from numpy import load
from pathlib import Path

# **1. Full-field flicker stimulus**

The data you are going to work with comes from extracellular recording of a retinal ganglion cell while stimulated with a full-field flicker, a stimulus consisting of a screen-wide presentation of contrast levels that changed with a given frequency. The contrast value of each presentation is calculated from a number that's been taken "randomly" from a (...).

![](fff.gif)

# **2. Loading the data**

In [ ]:
filepath = Path("")

data = load(filepath)


Now, the file just loaded has a number of variables stored, whose names can be retrieved in a list so:

In [ ]:
data.files

If we want to see the value of each variable, we retrieve it so:

In [ ]:
data["name_of_the_variable"]

If we want to be able to manipulate the variables, we can assign their values to new variables. Keeping the original name is the most logical.

In [ ]:
my_new_variable = data["name_of_the_variable"]

As with all things, there's a more straightforward way to update the file variables to our workspace:

In [ ]:
locals().update(data)

Let us go over what each of these variables represent, beginning with "volts". As its name suggests, it is the actual voltage recording, i.e., a list of numbers corresponding to the voltage values recorded throughout the experiment. Let's do a sanity check and see if we have spikes to begin with. To visualize the voltage trace, plot this variable.

In [ ]:
plt.plot(volts)

We won't be working further with the volts variable and will now focus on the other three: "spikes", "ttls" and "stim_rand_nums". "spikes" is a list of numbers representing timestamps –in seconds– of each occurrence of a spike. "ttls" is a list of timestamps of pulses that are generated to signal the presentation of the stimulus, so "ttls" is a list of the time points when the stimulus changed. "stim_rand_nums" is the sequence of "random" numbers that determined the contrast of each stimulus presentation, and for simplicity we will take them as a "measure" of contrast. Let's carry out another sanity check and confirm if, as should be expected already, the size of "ttls" and "stim_rand_nums" is the same. 

In [ ]:
###START CODE HERE###

###END CODE HERE###

Remember that we are going to collect the stimulus section preceding each spike. But, whatever the size of the sections, how much time do they represent? We must first consider that a given sequence of contrast values (random numbers) will correspond to as many stimulus presentations, so the time it took to show those contrast levels equals the number of stimulus presentation multiplied by the time elapsed from one stimulus presentation to the next. The former value we have in the desired size of our stimulus sections, whereas the latter we find by calculating the difference between any two consecutive values or, even better, the average of the differences between all pairs of consecutive values. 

In [ ]:
###START CODE HERE###
tlls[10] - ttls [9]

diff = np.diff(ttls)
avg_diff = diff.mean()

###END CODE HERE###

Since different cells integrate information over different stretches of time, we have to try with different time windows, where 0.5-2 seconds is a nice range. Once we have settled on a time window, the first thing we want to  to do is to find the first spike that occured so long after the stimulus presentation began, that we can already collect the first stimulus portion of the desired size.

In [ ]:
###START CODE HERE###
window = 15
for spike in spikes:
    if ttls[window] < spike:
        spike_idx = np.where(spikes == spike)[0][0]
        break
    else:
        pass
###FINISH CODE HERE

Now that we know with which spike to begin with, let's initialize the variable that will store the stimulus portions preceding each spike:

In [ ]:
stim_matrix = []

We are going to take all the spikes one by one (i.e., their timestamps) and take the corresponding stimulus portion preceding it, starting with the stimulus value at the instant immediately before the occurrence of the spike and stretching back by the value of the window previously defined:

In [ ]:
while spike_idx < len(spikes):
    
    idx = np.abs(ttls - spikes[spike_idx]).argmin()
    if ttls[idx] < spikes[spike_idx]:  
        stim_vect = stim_rand_nums[((idx+1)-window):idx+1]
    else:
        stim_vect = stim_rand_nums[(idx-window):idx]
    
    stim_matrix.append(stim_vect)
    spike_idx += 1
    
stim_matrix = np.asarray(stim_matrix)

We calculate the STA:

In [ ]:
average_stim = np.sum(stim_matrix, axis = 0)
sta = average_stim/len(stim_matrix)


Let's first plot the STA alone and see what we have:

In [ ]:
plt.plt(sta)

Add labels and an adequate x axis:

In [ ]:
###START CODE HERE###
time_limit_past = -avg_diff*window
x_ax = np.linspace(time_limit_past, 0, num = len(sta))
plt.plot(x_ax,sta)
plt.xlabel('Time (s)')
plt.ylabel('Contrast')